In [2]:
################################################
#################### Util ######################
################################################
import pickle
from collections import OrderedDict
from nltk.tokenize.casual import TweetTokenizer
import re

def get_vocab():
    try:
        with open('vocab.pickle', 'rb') as handle:
            return pickle.load(handle)
    except FileNotFoundError:
        vocab = OrderedDict()
        vocab['end of string'] = 0
        return vocab
    
tknzr = TweetTokenizer()
def tokenize(str):
    str = re.sub(r'(:[\w_]+:)', r'<\1>', str)
    return tknzr.tokenize(str)


In [52]:
################################################
################ Build Vocab ###################
################################################
inputFiles = ['answers_simple_small.txt', 'context_simple_small.txt'];

vocab = get_vocab()

for filePath in inputFiles:
    with open(filePath) as file:
        for line in file:
            tokens = tokenize(line)

            for token in tokens:
                if not (token in vocab):
                    vocab[token] = len(vocab)
                    
with open('vocab.pickle', 'wb') as file:
    pickle.dump(vocab, file, protocol=pickle.HIGHEST_PROTOCOL)
    print('{0} words in vocab'.format(len(vocab)))

638 words in vocab


In [81]:
################################################
################ encode words ##################
################################################
import os

input_files = ['answers_simple_small.txt', 'context_simple_small.txt'];

vocab = get_vocab()

for file_path in input_files:
    file_name, ext = os.path.splitext(file_path)
    
    with open(file_path) as input_file:
        output_file_path = file_name + '.encoded' + ext
        with open(output_file_path, 'w') as output_file:
            max_tokens = 0
            for line in input_file:
                max_tokens = max(max_tokens, len(tokenize(line)))
            input_file.seek(0)
            
            for line in input_file:
                encoded_tokens = [str(vocab[token]) for token in tokenize(line)]
                encoded_tokens += ['0'] * (max_tokens - len(encoded_tokens))
                output_file.write(' '.join(encoded_tokens) + '\n')
            print('Encoded "{0}" to "{1}".'.format(file_path, output_file_path))

Encoded "answers_simple_small.txt" to "answers_simple_small.encoded.txt".
Encoded "context_simple_small.txt" to "context_simple_small.encoded.txt".


In [22]:
################################################
################# train model ##################
################################################
import numpy as np
from keras.models import Model
from keras.layers import Input, GRU, Dense, Embedding
from keras.utils import to_categorical
import pickle

vocab = get_vocab()

model_input_file = 'context_simple_small.encoded.txt'
model_output_file = 'context_simple_small.encoded.txt'
THOUGHT_VECTOR_SIZE = 100

encoder_input_data = np.loadtxt(model_input_file)
decoder_target_data = np.loadtxt(model_output_file)
decoder_input_data = decoder_target_data[:, :-1]
decoder_input_data = np.insert(decoder_input_data, 0, values=0, axis=1)
decoder_target_one_hot = to_categorical(decoder_target_data, len(vocab))


#Create layers
encoder_input_layer = Input(shape=(None,))
encoder_embedding_layer = Embedding(len(vocab), THOUGHT_VECTOR_SIZE)
encoder_gru_layer = GRU(THOUGHT_VECTOR_SIZE, return_state=True)

decoder_input_layer = Input(shape=(None,))
decoder_embedding_layer = Embedding(len(vocab), THOUGHT_VECTOR_SIZE)
decoder_gru_layer = GRU(THOUGHT_VECTOR_SIZE, return_sequences=True)
decoder_dense_layer = Dense(len(vocab), activation='softmax')


#connect network
encoder = encoder_embedding_layer(encoder_input_layer)
encoder, encoder_state = encoder_gru_layer(encoder)

decoder = decoder_embedding_layer(decoder_input_layer)
decoder = decoder_gru_layer(decoder, initial_state=encoder_state)
decoder = decoder_dense_layer(decoder)


model = Model([encoder_input_layer, decoder_input_layer], decoder)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_one_hot,
          batch_size=32,
          epochs=10,
          validation_split=0.2)

network_config = {
    'vocab_size': len(vocab),
    'thought_vector_size': THOUGHT_VECTOR_SIZE,
    'sequence_length': encoder_input_data.shape[1],
    'weights': {
        'encoder_embedding': encoder_embedding_layer.get_weights(),
        'encoder_gru': encoder_gru_layer.get_weights(),
        'decoder_embedding': decoder_embedding_layer.get_weights(),
        'decoder_gru': decoder_gru_layer.get_weights(),
        'decoder_dense': decoder_dense_layer.get_weights()        
    }
}

with open('network_config.pickle', 'wb') as file:
    pickle.dump(network_config, file)

print('saved network config to "{}". Vocab size: {}. Thought vector size: {}. Sequence length: {}.'
    .format(
        'network_config.pickle', 
        network_config['vocab_size'], 
        network_config['thought_vector_size'], 
        network_config['sequence_length']
    )
)  

Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 5s 31ms/step - loss: 6.3174 - val_loss: 5.9948
Epoch 2/10
160/160 [==============================] - 3s 17ms/step - loss: 4.1978 - val_loss: 3.9734
Epoch 3/10
160/160 [==============================] - 2s 13ms/step - loss: 3.1506 - val_loss: 3.9060
Epoch 4/10
160/160 [==============================] - 2s 13ms/step - loss: 3.0615 - val_loss: 3.8131
Epoch 5/10
160/160 [==============================] - 2s 14ms/step - loss: 3.0027 - val_loss: 3.8123
Epoch 6/10
160/160 [==============================] - 2s 13ms/step - loss: 3.0176 - val_loss: 3.7823
Epoch 7/10
160/160 [==============================] - 2s 13ms/step - loss: 2.8914 - val_loss: 4.0818
Epoch 8/10
160/160 [==============================] - 2s 13ms/step - loss: 2.9155 - val_loss: 3.7380
Epoch 9/10
160/160 [==============================] - 2s 14ms/step - loss: 2.8510 - val_loss: 3.9573
Epoch 10/10
160/160 [=========================

In [30]:
################################################
############## inference models ################
################################################

with open('network_config.pickle', 'rb') as file:
    config = pickle.load(file)
    
#Encoder
encoder_input_layer = Input(shape=(None,))
encoder_embedding_layer = Embedding(config['vocab_size'], config['thought_vector_size'])
encoder_gru_layer = GRU(config['thought_vector_size'], return_state=True)

encoder = encoder_embedding_layer(encoder_input_layer)
encoder, encoder_state = encoder_gru_layer(encoder)

encoder_model = Model([encoder_input_layer], encoder_state)
encoder_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
encoder_embedding_layer.set_weights(config['weights']['encoder_embedding'])
encoder_gru_layer.set_weights(config['weights']['encoder_gru'])

#Decoder
decoder_input_layer = Input(shape=(None,))
decoder_thought_vector_input_layer = Input(shape=(config['thought_vector_size'],))
decoder_embedding_layer = Embedding(config['vocab_size'], config['thought_vector_size'])
decoder_gru_layer = GRU(config['thought_vector_size'], return_sequences=True)
decoder_dense_layer = Dense(config['vocab_size'], activation='softmax')

decoder = decoder_embedding_layer(decoder_input_layer)
decoder = decoder_gru_layer(decoder, initial_state=decoder_thought_vector_input_layer)
decoder = decoder_dense_layer(decoder)

decoder_model = Model([decoder_input_layer, decoder_thought_vector_input_layer], decoder)
decoder_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
decoder_embedding_layer.set_weights(config['weights']['decoder_embedding'])
decoder_gru_layer.set_weights(config['weights']['decoder_gru'])
decoder_dense_layer.set_weights(config['weights']['decoder_dense'])

print('finished creating models')

finished creating models


In [49]:
vocab = get_vocab()

def reply(input_str):
    tokens = tokenize(input_str)
    encoded_tokens = [vocab[token] for token in tokenize(input_str)]
    encoded_tokens += [0] * (config['sequence_length'] - len(encoded_tokens))
    
    # Encode the input as state vectors.
    thought_vector = encoder_model.predict(np.array([encoded_tokens]))[0]
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, config['vocab_size']))
    # Populate the first character of target sequence with a 0 (end of string). Should maybe use something else for this
    target_seq[0, 0, 0] = 1.

    output_tokens, h = decoder_model.predict(
        np.array(target_seq + [[thought_vector]]))
    
    return
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h = decoder_model.predict(
            [target_seq] + thought_vector)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

reply('who is in your family?')

ValueError: operands could not be broadcast together with shapes (1,1,638) (1,1,100) 